## Vereadores mais votados do município do Rio de Janeiro por zona

Esse programa lê o banco de dados do TSE para votações estaduais de 2016 e seleciona os vereadores de um determinado município, filtrando apenas as variáveis desejadas, e gera um novo arquivo *.csv*.

Esses dados podem ser encontrados no repositório de dados eleitorais do TSE:
**http://www.tse.jus.br/eleicoes/estatisticas/repositorio-de-dados-eleitorais-1/repositorio-de-dados-eleitorais**

A lista de zonas eleitorais do Estado do Rio por bairro pode ser encontrada no site do TRE:
http://www.tre-rj.jus.br/site/eleicoes/2016/arquivos/porBairroZon.xls

In [9]:
# Importa os dados dos candidatos do estado do Rio em 2016.
# Inicialmente são 30 as variáveis no banco de dados para cada linha.

import csv
import pandas as pd
import os
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# Define como delimitador o ponto e vírgula
# Sem isso o Python lê a linha inteira do CSV com um índice único numa lista
csv.register_dialect('pVirgula', delimiter=';')
csv.register_dialect('virgula', delimiter=',')

# Importa os dados dos candidatos do estado do Rio em 2016
with open ('src/votacao_candidato_munzona_2016_RJ.csv', 'r', encoding="latin-1") as vot_f:
    leitor = csv.reader(vot_f, dialect='pVirgula')
    dados = list(leitor)


In [3]:
# lista com os rótulos dos dados
cols = ['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO', 
           'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 
           'DT_ELEICAO', 'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE',
           'CD_MUNICIPIO', 'NM_MUNICIPIO', 'NR_ZONA', 'CD_CARGO', 'DS_CARGO',
           'SQ_CANDIDATO', 'NR_CANDIDATO', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO',
           'NM_SOCIAL_CANDIDATO', 'CD_SITUACAO_CANDIDATURA', 
           'DS_SITUACAO_CANDIDATURA', 'CD_DETALHE_SITUACAO_CAND', 
           'DS_DETALHE_SITUACAO_CAND', 'TP_AGREMIACAO', 'NR_PARTIDO', 
           'SG_PARTIDO', 'NM_PARTIDO', 'SQ_COLIGACAO', 'NM_COLIGACAO', 
           'DS_COMPOSICAO_COLIGACAO', 'CD_SIT_TOT_TURNO', 'DS_SIT_TOT_TURNO', 
           'ST_VOTO_EM_TRANSITO', 'QT_VOTOS_NOMINAIS']

drop_cols = ['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
         'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO',
         'DT_ELEICAO', 'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE',
         'CD_MUNICIPIO','CD_CARGO', 'SQ_CANDIDATO', 'NR_CANDIDATO',
         'NM_CANDIDATO', 'NM_SOCIAL_CANDIDATO', 'CD_SITUACAO_CANDIDATURA',
         'DS_SITUACAO_CANDIDATURA', 'CD_DETALHE_SITUACAO_CAND',
         'DS_DETALHE_SITUACAO_CAND', 'TP_AGREMIACAO', 'NM_PARTIDO',
         'SQ_COLIGACAO', 'NM_COLIGACAO', 'DS_COMPOSICAO_COLIGACAO',
         'CD_SIT_TOT_TURNO',  'ST_VOTO_EM_TRANSITO']

# Lista vazia para receber os candidatos
rows = []

# Loop para selecionar apenas os candidatos a vereador do município do rio
# que foram eleitos
for linhas in dados:
    if linhas[14] == "RIO DE JANEIRO" and linhas[17] == "Vereador" and linhas[35] != "SUPLENTE" and linhas[35] != "NÃO ELEITO":
        rows.append(linhas)
    else:
        continue

# Utilizando o Pandas, adiciona os rótulos das colunas e exclui as
# colunas indesejadas
ver = pd.DataFrame(rows)
ver.columns = cols
ver.drop(drop_cols, axis=1, inplace=True)

In [4]:
# Experimentações de análises com Pandas
zonas = ver.groupby(['NR_ZONA']).count()
zonas

# zona = ver.loc[ver['NR_ZONA'] == '1']
# zona.sort_values(['QT_VOTOS_NOMINAIS'], ascending=True)

,NM_MUNICIPIO,DS_CARGO,NM_URNA_CANDIDATO,NR_PARTIDO,SG_PARTIDO,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS
NR_ZONA,,,,,,,
1,51,51,51,51,51,51,51
10,51,51,51,51,51,51,51
11,51,51,51,51,51,51,51
117,51,51,51,51,51,51,51
118,51,51,51,51,51,51,51
119,51,51,51,51,51,51,51
12,51,51,51,51,51,51,51
120,51,51,51,51,51,51,51
121,51,51,51,51,51,51,51


In [5]:
# Salva os vereadores selecionados em um novo arquivo .csv via Pandas
exportar_csv = ver.to_csv(r'output\votacao_filtro_munzona_2016_RJ.csv', encoding='latin-1', index=None, header=True)
#exportar_js = ver.to_json(r'output\ver3_votacao_filtro_munzona_2016_RJ.json')

vot_f.close()

In [55]:
ver_rj = pd.read_csv("output/votacao_filtro_munzona_2016_RJ.csv", encoding="latin-1")
vr = pd.DataFrame(ver_rj)
cand = vr.groupby(['NM_URNA_CANDIDATO']).count()

vr.head(1)

,NM_MUNICIPIO,NR_ZONA,DS_CARGO,NM_URNA_CANDIDATO,NR_PARTIDO,SG_PARTIDO,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS
0,RIO DE JANEIRO,214,Vereador,TARCÍSIO MOTTA,50,PSOL,ELEITO POR QP,1594
